In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#import data
df = pd.read_csv('../../long_short_local/raw_data/cleaned_data.csv')

In [3]:
df.describe()

,ALB_ZBRA,NRG_CPRT,DE_BXP,NRG_STE,NRG_XRAY,SO_NLOK,PEG_O
count,497.000000,497.000000,497.000000,497.000000,497.000000,497.000000,497.000000
mean,0.439148,0.296925,3.242148,0.178814,0.728099,2.664304,0.960830
std,0.134129,0.038166,0.309264,0.017806,0.170586,0.296301,0.047787
min,0.279760,0.219809,2.513787,0.147486,0.464028,2.094566,0.829956
25%,0.336085,0.270483,3.054503,0.164950,0.624090,2.436779,0.928549
50%,0.403014,0.290112,3.224409,0.175372,0.691772,2.649792,0.960627
75%,0.464571,0.316606,3.435012,0.189553,0.742545,2.861494,0.994654
max,0.822204,0.425462,4.280950,0.231046,1.230987,3.405268,1.094158


## Splitting Data in Train and Validation

In [4]:
# R
def split_train_val(df):
    length_data = len(df)    
    split_ratio = 0.7           # %70 train + %30 validation
    length_train = round(length_data * split_ratio)  
    length_validation = length_data - length_train
    #print("Data length :", length_data)
    #print("Train data length:", length_train)
    #print("Validation data lenth:", length_validation)
    
    train_data = df[:length_train].iloc[:,:2] 
    train_data['Date'] = pd.to_datetime(train_data['Date'])  # converting to date time object

    validation_data = df[length_train:].iloc[:,:2]
    validation_data['Date'] = pd.to_datetime(validation_data['Date'])  # converting to date time object
    
    return train_data, validation_data, length_train, length_validation

## Creating Train Dataset from Train split

In [5]:
# R
# create dataset from train split
def train_split(train_data):
    dataset_train = train_data.iloc[:, 1].values
    # Change 1d array to 2d array
    # Changing shape from (1692,) to (1692,1)
    dataset_train = np.reshape(dataset_train, (-1,1))
    #dataset_train.shape
    dataset_train_scaled = dataset_train
    
    return dataset_train_scaled

## Creating X_train and y_train from Train data 

We have train data composed of ratios over days

So, it has 1184 ratios corresponding 1184 days

My aim is to predict the ratio (closing) of the next day.

I can use a time step of 2 days.

I will pick first 2 ratios (day 0 to 2), 1st 2 ratio will be in 
X_train data

Then predict the ratio of 3rd day; and 3rd ratio will be in y_train data

Again, i will pick ratios from 1 to 3, those will be in X_train data

Then predict the next days ratio, 4th ratio will be in y_train data



In [6]:
def create_x_y_train(df, length_train):
    X_train = []
    y_train = []

    time_step = 20 #change that?

    for i in range(time_step, length_train):
        X_train.append(dataset_train_scaled[i-time_step:i,0:1])
        y_train.append(dataset_train_scaled[i,0:1])

    # convert list to array
    X_train, y_train = np.array(X_train), np.array(y_train)
    
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
    y_train = np.reshape(y_train, (y_train.shape[0],1))
    
    X_train = X_train[:int(X_train.shape[0]*0.95)]
    X_val = X_train[int(X_train.shape[0]*0.95):]
    y_train = y_train[:int(y_train.shape[0]*0.95)]
    y_val = y_train[int(y_train.shape[0]*0.95):]
    
    return X_train, X_val, y_train, y_val, time_step

## Creating Test Dataset from Validation Data

In [7]:
# R
#Converting array and scaling
def create_x_y_test(validation_data, length_validation, time_step):
    dataset_validation = validation_data.iloc[:,1].values  # getting "Ratio" column and converting to array
    dataset_validation = np.reshape(dataset_validation, (-1,1))  # converting 1D to 2D array
    #scaled_dataset_validation =  scaler.fit_transform(dataset_validation)  # scaling  values to between 0 and 1
    scaled_dataset_validation = dataset_validation
    
    X_test = []
    y_test = []

    for i in range(time_step, length_validation):
        X_test.append(scaled_dataset_validation[i-time_step:i,0])
        y_test.append(scaled_dataset_validation[i,0])
        
    # Converting to array
    X_test, y_test = np.array(X_test), np.array(y_test)
    X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))  # reshape to 3D array
    y_test = np.reshape(y_test, (-1,1))  # reshape to 2D array
    
    return X_test, y_test

## Implemenation refactored

In [8]:
# R
# split into train/test
one_ratio_df = pd.DataFrame(df[['Date', 'ALB_ZBRA']])#[ratio]
train_data, validation_data, length_train, length_validation = split_train_val(one_ratio_df)
# call train_split
dataset_train_scaled = train_split(train_data)
# create X_train, y_train
X_train, X_val, y_train, y_val, time_step = create_x_y_train(dataset_train_scaled, length_train)
# create X_test, y_test
X_test, y_test = create_x_y_test(validation_data, length_validation, time_step)

## Creating LSTM Model

In [ ]:
y_train.min()

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
from keras.layers import LSTM
from keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import Dense


es = EarlyStopping(patience=20, restore_best_weights=True)
model_lstm = Sequential()

model_lstm.add(LSTM(20,return_sequences=False,input_shape = (X_train.shape[1],1))) #64 lstm neuron block
model_lstm.add(Dense(32))
model_lstm.add(Dense(1))

model_lstm.compile(loss = "mape", optimizer = "rmsprop", metrics = ["mae", "mape"])
history2 = model_lstm.fit(X_train, y_train, epochs = 400, batch_size = 64,validation_data = (X_val, y_val),callbacks=[es])

## Evaluating LSTM Model

In [ ]:
model_lstm.evaluate(X_test, y_test)

In [ ]:
plt.figure(figsize =(10,5))
plt.plot(history2.history["loss"])
plt.xlabel("Epochs")
plt.ylabel("Losses")
plt.title("LSTM model, Accuracy vs Epoch")
plt.show()

In [ ]:
plt.subplots(figsize =(30,12))
plt.plot(model_lstm.predict(X_test), label = "y_pred_of_test", c = "orange" )
plt.plot(y_test, label = "y_test", color = "g")
plt.xlabel("Days")
plt.ylabel("Ratio")
plt.title("LSTM model, Predictions with input X_test vs y_test")
plt.legend()
plt.show()

## Future ratio prediction

In [ ]:
#last day in our data?
df.iloc[-1]

### We can predict the ratio for the day after 2022-08-23--> for 2022-08-24.
We will use last ratios as input of our model for this prediction.

In [ ]:
X_input = df.iloc[-time_step:].ALB_ZBRA.values             # getting last rows and converting to array
X_input = (X_input.reshape(-1,1))     # converting to 2D array and scaling
X_input = np.reshape(X_input, (1,time_step,1))                    # reshaping: converting to 3D array
print("Shape of X_input :", X_input.shape)
X_input

In [ ]:
LSTM_prediction = model_lstm.predict(X_input)
print("LSTM, prediction for 2022-08-24:", LSTM_prediction[0,0])